In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from hr_db_scripts.main_db_script import get_db_table_as_df
from main_db_script import db_filename
import sqlite3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
def indicies_where_daily_rain_exceeds_threshold(df, threshold):
    rd = df[[c for c in df.columns if c.startswith('rd')]]
    ind_abv_threshold = rd[rd.max(axis=1)>0.1].index
    return ind_abv_threshold

In [3]:
flood_locations = get_db_table_as_df('flood_locations', dbfilename=db_filename)
flood_events = get_db_table_as_df('flood_events', date_col=['event_date', 'dates'], dbfilename=db_filename)
for_model = get_db_table_as_df('for_model', date_col=['event_date'], dbfilename=db_filename)
daily_obs = get_db_table_as_df('nor_daily_observations', date_col=['Datetime'], dbfilename=db_filename)

In [4]:
print flood_locations.shape
flood_locations = flood_locations[flood_locations.in_hague==1]
print flood_locations.shape
flood_locations.head()

(1496, 24)
(101, 24)


,index,location,xcoord,ycoord,count,twi,elev_1,dist_to_ba,imp,dist_to_wa,...,Invert_Ele,Year_Insta,Pipe_Geome,Horizontal,Vertical_D,Pipe_Lengt,Pipe_Mater,Year_Ins_1,Condition,flood_pt
1017,1017,BALDWIN AVENUE & STOCKLEY GARDENS,1.212808e+07,3.482139e+06,1,5.32951,1.89221,61.4517,133575,2749.43,...,2.55,1937,CR,18.0,0.0,9.849103,CAP,1937,None,1
1032,1032,1100 BLOCK OF COLLEY AVENUE,1.212666e+07,3.482094e+06,1,3.18615,2.64143,475.5410,133362,2259.05,...,4.27,1922,CR,24.0,0.0,30.000000,CONC,1922,None,1
1038,1038,300 BLOCK OF SHIRLEY AVENUE,1.212926e+07,3.481753e+06,5,3.81674,2.27047,21.0453,108929,3124.06,...,5.80,1937,None,0.0,0.0,32.650009,None,0,None,1
1041,1041,...,0.000000e+00,0.000000e+00,0,9.11032,1.70383,21.8410,132510,2456.79,...,4.97,1937,CR,12.0,0.0,40.000000,CLAY,1937,None,0
1047,1047,BLOW STREET & W PRINCESS ANNE ROAD,1.212779e+07,3.481628e+06,1,10.89350,1.59764,18.5604,130618,2119.38,...,3.22,1937,None,0.0,0.0,9.848905,None,0,None,1


In [5]:
daily_obs.set_index('Datetime', inplace=True)

In [6]:
print daily_obs.shape
rain_threshold = 0.1
rd_abv_thresh = indicies_where_daily_rain_exceeds_threshold(daily_obs, rain_threshold)

(2558, 113)


In [7]:
daily_obs = daily_obs.loc[rd_abv_thresh,:]
print daily_obs.shape
daily_obs.head()

(782, 113)


,WDF2-3,WDF2-19,WDF2-20,WSF2-3,WSF2-19,WSF2-20,AWDR-19,AWND-19,AWND-20,WGF6-18,...,r3d-1,r3d-2,r3d-7,r3d-11,r3d-12,r3d-13,r3d-14,r3d-15,r3d-16,r3d-21
Datetime,,,,,,,,,,,,,,,,,,,,,
2010-01-04,NaN,330.0,310.0,NaN,16.1,21.0,306.0,7.2,9.8,9.317000,...,NaN,NaN,5.000000e-02,4.000000e-02,0.06,0.04,1.000000e-02,0.01,0.02,3.000000e-02
2010-01-05,NaN,310.0,300.0,NaN,15.0,17.0,292.0,6.0,8.5,8.925750,...,NaN,NaN,1.000000e-01,1.100000e-01,0.00,0.13,0.000000e+00,0.00,0.00,0.000000e+00
2010-01-14,NaN,30.0,220.0,NaN,8.9,8.9,216.0,4.5,3.8,3.701167,...,NaN,NaN,1.387779e-17,1.387779e-17,0.00,0.00,-1.040834e-17,0.00,0.00,3.469447e-18
2010-01-17,NaN,250.0,240.0,NaN,21.0,25.9,176.0,8.5,8.1,9.813500,...,NaN,NaN,1.387779e-17,1.387779e-17,0.12,0.00,-1.040834e-17,0.00,0.00,3.469447e-18
2010-01-19,NaN,240.0,250.0,NaN,17.0,14.1,229.0,7.8,6.3,5.872092,...,NaN,NaN,8.200000e-01,1.387779e-17,0.80,0.95,7.300000e-01,0.55,0.71,8.300000e-01


In [8]:
flood_events.head()

,location,eventType,event_name,event_date,dates,times
0,300 BLOCK OF SHIRLEY AVENUE,Flooded street,Nicole-2010-09-30,2010-09-30,2010-09-30,07:32:47.000
1,LLEWELLYN AVENUE & SHIRLEY AVENUE,Flooded street,Nicole-2010-09-30,2010-09-30,2010-09-30,07:33:19.000
2,LLEWELLYN AVENUE & W PRINCESS ANNE ROAD,Flooded street,Nicole-2010-09-30,2010-09-30,2010-09-30,07:33:48.000
3,900 BLOCK OF TIDEWATER DRIVE,Flooded street,Nicole-2010-09-30,2010-09-30,2010-09-30,08:29:00.000
4,CAMPOSTELLA ROAD & E BRAMBLETON AVENUE,Flooded street,Nicole-2010-09-30,2010-09-30,2010-09-30,08:29:38.000


In [9]:
for_model.set_index('event_date', inplace=True)
for_model.shape

(2507, 116)

In [10]:
for_model = for_model.loc[indicies_where_daily_rain_exceeds_threshold(for_model, rain_threshold),:]
print for_model.shape
for_model.head()

(758, 116)


,event_name,num_flooded,AWDR-19,AWND-19,AWND-20,WDF2-19,WDF2-20,WDF2-3,WDF6-18,WGF6-18,...,rhrmx-13,rhrmx-14,rhrmx-15,rhrmx-16,rhrmx-2,rhrmx-21,rhrmx-7,td_av-17,td_av-18,flooded
event_date,,,,,,,,,,,,,,,,,,,,,
2010-09-30,Nicole-2010-09-30,101.0,291.666667,11.600,12.600000,83.333333,160.000000,NaN,NaN,NaN,...,1.40,1.17,0.90,1.31,NaN,1.49,1.43,1.391597,1.531639,1
2010-12-16,Snow-2010-12-16,2.0,213.000000,2.900,2.500000,210.000000,200.000000,NaN,NaN,NaN,...,0.05,0.12,0.06,0.12,NaN,0.03,0.11,-0.215292,-0.202708,1
2011-08-27,Irene-2011-08-27,110.0,133.000000,13.975,14.725000,130.000000,187.500000,NaN,133.611372,13.953947,...,1.12,0.93,1.02,1.30,NaN,1.10,1.03,1.038927,1.139156,1
2012-10-28,Sandy-2012-10-28,105.0,123.000000,23.800,24.000000,20.000000,16.666667,107.628500,110.730125,20.061461,...,0.40,0.38,0.52,0.43,NaN,0.48,0.23,2.705250,2.797889,1
2013-10-09,Heavy Rain-2013-10-09,36.0,25.666667,20.800,21.566667,36.666667,26.666667,52.956282,41.512500,15.315069,...,0.47,29.86,0.56,0.40,NaN,0.46,0.50,2.212722,2.281167,1


In [11]:
print flood_locations.shape
flood_locations = flood_locations[flood_locations['count']>1]
flood_locations.shape

(101, 24)


(33, 24)

In [12]:
lst = []
for index, row in flood_locations.iterrows():
    l = row['location'].strip()
    loc_data = daily_obs.copy()
    loc_data['flooded'] = 0
    if l != '':
        event_dates = flood_events[flood_events['location'] == l]['event_date']
        all_fld_dates = flood_events[flood_events['event_date'].isin(event_dates)]['dates']
        all_fld_dates = np.append(all_fld_dates, event_dates)
        all_fld_dates = pd.to_datetime(np.unique(all_fld_dates))

        fld_data = for_model[for_model.index.isin(event_dates)]
    
        loc_data = loc_data[~loc_data.index.isin(all_fld_dates)]
        loc_data = pd.concat([loc_data, fld_data])
    
        # add geog data
    for k in row.keys():
        loc_data[k] = row[k]
    lst.append(loc_data)
#         print l
#         print loc_data.head()
        

In [13]:
all_locations = pd.concat(lst)
all_locations.reset_index(inplace=True)

In [14]:
all_locations.shape
all_locations.head()

,level_0,AWDR-19,AWND-19,AWND-20,WDF2-19,WDF2-20,WDF2-3,WDF6-18,WGF6-18,WSF2-19,...,Invert_Ele,Year_Insta,Pipe_Geome,Horizontal,Vertical_D,Pipe_Lengt,Pipe_Mater,Year_Ins_1,Condition,flood_pt
0,2010-01-04,306.0,7.2,9.8,330.0,310.0,NaN,324.969875,9.317000,16.1,...,5.8,1937,None,0.0,0.0,32.650009,None,0,None,1
1,2010-01-05,292.0,6.0,8.5,310.0,300.0,NaN,317.637125,8.925750,15.0,...,5.8,1937,None,0.0,0.0,32.650009,None,0,None,1
2,2010-01-14,216.0,4.5,3.8,30.0,220.0,NaN,203.150042,3.701167,8.9,...,5.8,1937,None,0.0,0.0,32.650009,None,0,None,1
3,2010-01-17,176.0,8.5,8.1,250.0,240.0,NaN,186.049208,9.813500,21.0,...,5.8,1937,None,0.0,0.0,32.650009,None,0,None,1
4,2010-01-19,229.0,7.8,6.3,240.0,250.0,NaN,241.470753,5.872092,17.0,...,5.8,1937,None,0.0,0.0,32.650009,None,0,None,1


In [15]:
del all_locations['level_0']
all_locations.tail()

,AWDR-19,AWND-19,AWND-20,WDF2-19,WDF2-20,WDF2-3,WDF6-18,WGF6-18,WSF2-19,WSF2-20,...,Invert_Ele,Year_Insta,Pipe_Geome,Horizontal,Vertical_D,Pipe_Lengt,Pipe_Mater,Year_Ins_1,Condition,flood_pt
25633,243.0,18.300,14.800,220.0,20.000000,186.571458,NaN,NaN,32.000000,28.000,...,3.42,1924,CR,15.0,0.0,300.0,CONC,1924,None,1
25634,228.0,6.900,5.600,230.0,20.000000,199.806486,NaN,NaN,16.100000,12.100,...,3.42,1924,CR,15.0,0.0,300.0,CONC,1924,None,1
25635,226.0,6.500,9.400,290.0,300.000000,222.198528,NaN,NaN,15.000000,25.100,...,3.42,1924,CR,15.0,0.0,300.0,CONC,1924,None,1
25636,123.0,23.800,24.000,20.0,16.666667,107.628500,110.730125,20.061461,37.633333,39.000,...,3.42,1924,CR,15.0,0.0,300.0,CONC,1924,None,1
25637,58.5,16.775,15.925,57.5,60.000000,4.148030,65.646875,11.669104,24.675000,23.725,...,3.42,1924,CR,15.0,0.0,300.0,CONC,1924,None,1


In [16]:
x_train, x_test, y_train, y_test = train_test_split(all_locations.index, all_locations.flooded, test_size=0.3, stratify=all_locations.flooded)

In [17]:
x_train_fld = all_locations.index[(all_locations.index.isin(x_train)) & (all_locations.flooded==True)]
# x_train_fld = all_locations.index[all_locations.flooded==True]
x_train_nfld = all_locations.index[(all_locations.index.isin(x_train)) & (all_locations.flooded==False)]
x_train_nfld_sampled = np.random.choice(x_train_nfld, size=len(x_train_fld)*5)
x_train_combined = np.concatenate((x_train_fld, x_train_nfld_sampled))
train_data = all_locations.loc[x_train_combined, :]
train_data = all_locations.loc[x_train, :]

In [18]:
test_data = all_locations.loc[x_test, :]

In [19]:
print [a.shape for a in [x_train_fld, x_train_nfld,x_train_combined,train_data,test_data]]

[(73L,), (17873L,), (438L,), (17946, 140), (7692, 140)]


In [20]:
con = sqlite3.connect(db_filename)
train_data.to_sql(con=con, name='train_geog_data_hg', if_exists='replace')
test_data.to_sql(con=con, name='test_geog_data_hg', if_exists='replace')